### Card Simulator

#### Rules

- For now, ace is always low
- Rank hands:
    - Straight flush
    - 4 of a kind
    - Flush
    - Straight
    - 3 of a kind
    - 2 of a kind
    - high card
    - after that it's a tie
    
- Cards need to have an internal representaton, and a human-readable representation

In [1]:
from __future__ import annotations
import random

class Hand:
    # Hand object should be
    # a collection of cards
    # and determine whether there are 
    # pairs, straights, etc.
    
    def __init__(self,
                 cards: list[Card]
                ):
        self.__validate_hand(cards)
        self.__cards = cards
        self.count_faces_suits()
        self.__cards_sorted = sorted(self.__cards)
        
        self.__create_rules()
        
        return 
    
    def __validate_hand(self, 
                        cards: list[Card]):
        for i in range(len(cards)):
            for j in range(i + 1, len(cards)):
                if cards[i] == cards[j]:
                    raise ValueError(f'Hand contains two of: {cards[i]}')
    
    def __create_rules(self):
        self.__RULES = {'straight_flush' : self.get_straight_flush,
                             'four_kind' : self.get_quadruples,
                                 'flush' : self.get_flush,
                              'straight' : self.get_straight,
                            'three_kind' : self.get_triples,
                              'two_kind' : self.get_pairs,
                             'high_card' : self.get_high_card
                  }
    
    def get_best_hand(self) -> tuple[int]: # rank of the hand (low wins)
        
        for rank, (hand_name, func) in enumerate(self.__RULES.items()):
            # return when we get a not None response
            output = func()
            if all(output):
                return rank, hand_name, output
        return None
    
    def __repr__(self):
        return 'Hand with:\n' + str(self.__cards)
    
    def count_faces_suits(self):
        self.__face_counts = {}
        self.__suit_counts = {}
        
        for card in self.__cards:
            face = card.get_face_num()
            suit = card.get_suit_name()
            
            if face in self.__face_counts:
                self.__face_counts[face] += 1
            else:
                self.__face_counts[face] = 1
            
            if suit in self.__suit_counts:
                self.__suit_counts[suit] += 1
            else:
                self.__suit_counts[suit] = 1
        return
    
    def get_face_counts(self):
        return self.__face_counts.copy()
    
    def get_suit_counts(self):
        return self.__suit_counts.copy()
    
    def get_high_card(self):
        return (self.__cards_sorted[-1].get_face_num(), )
    
    def get_flush(self
                 ) -> tuple[int, # High card 
                            str  # Suit name
                           ]:
        for suit, count in self.__suit_counts.items():
            if count >= 5:
                high_card = 0
                for card in self.__cards:
                    if card.get_suit_name() == suit:
                        high_card = max(high_card, card.get_face_num())
                return high_card, suit
        return None, None
    
    def get_straight(self) -> int:
        # Compte the difference between the
        # consecutive sorted cards
        diffs = []
        for i in range(len(self.__cards_sorted)-1):
            diffs.append(self.__cards_sorted[i+1] - self.__cards_sorted[i])
        
        # We have a straight if the list of differences
        # contains 4 ones, not separated by anything other
        # than zeros
        ones = 0
        idx_high_card = -1
        for idx,val in enumerate(diffs):
            if val == 1:
                ones += 1
                if ones >= 4:
                    idx_high_card = idx
            elif (val == 0):
                pass
            else:
                ones = 0
                
        if idx_high_card == -1:
            return (None, )
        else:
            return (self.__cards_sorted[idx_high_card+1].get_face_num(), )
    
    def get_straight_flush(self) -> int:
        _, suit = self.get_flush()
        
        face = (None,)
        # if suit is not none, make a new hand containing cards from that suit
        if suit is not None:
            cards = []
            for card in self.__cards:
                if card.get_suit_name() == suit:
                    cards.append(card)
            hand = Hand(cards)
            # Check if the subset contains a straight
            face = hand.get_straight()
        return face
    
    def get_mults(self, n:int) -> list[str]:
        mults = []
        for face, count in self.__face_counts.items():
            if count == n:
                mults.append(face)
        if len(mults) == 0:
            return (None, )
        else:
            return (mults, )
    
    def get_pairs(self) -> list[str]:
        return self.get_mults(2)
    
    def get_triples(self) -> list[str]:
        return self.get_mults(3)
    
    def get_quadruples(self) -> list[str]:
        return self.get_mults(4)
    
    def __gt__(self, other):
        rank1, _, output1 = self.get_best_hand()
        rank2, _, output2 = other.get_best_hand()

        if rank1 == rank2:
            out1 = output1[0]
            out2 = output2[0]
            if type(out1) == list:
                out1 = max(out1)
                out2 = max(out2)
            return out1 > out2
            
        else:
            return rank1 < rank2
    
    def __eq__(self, other):
        rank1, _, output1 = self.get_best_hand()
        rank2, _, output2 = other.get_best_hand()
        
        if rank1 == rank2:
            out1 = output1[0]
            out2 = output2[0]
            if type(out1) == list:
                out1 = max(out1)
                out2 = max(out2)
            return out1 == out2
            
        else:
            return False

class Deck:
    def __init__(self):
        self.fill_deck()
        self.shuffle()
        return
    
    def fill_deck(self):
        self.__cards = [Card(n) for n in range(52)]
        return
    
    def shuffle(self,
                restore_deck: bool = True):
        if restore_deck:
            self.fill_deck()
        # A very unrealistic shuffle
        random.shuffle(self.__cards)
        return
    
    def deal(self, 
             num_cards: int
            ) -> Hand:
        
        cards = []
        
        if num_cards > len(self.__cards):
            raise ValueError("Not enough cards left in the deck.")
        
        while len(cards) < num_cards:
            cards.append(self.__cards.pop())
        
        return Hand(cards)

class Card:
    
    __SUITS = ['Clubs', 
               'Diamonds',
               'Hearts',
               'Spades']
    
    __FACES = {n+1:n+1 for n in range(13)}
    __FACES[1]  = 'Ace'
    __FACES[11] = 'Jack'
    __FACES[12] = 'Queen'
    __FACES[13] = 'King'
    
    __FACE_NUMS = {}
    __FACE_NUMS['Ace']   = 1
    __FACE_NUMS['Jack']  = 11
    __FACE_NUMS['Queen'] = 12
    __FACE_NUMS['King']  = 13
    
    def __init__(self, 
                 value: int = None,
                 suit:  str = None,
                 face:  str = None
                ):
        if suit is not None:
            suit = suit.capitalize()
        self.__validate_input(value, suit, face)
        
        if face is not None:
            if face in self.__FACE_NUMS:
                value = self.__FACE_NUMS[face]
            else:
                value = int(face)
        if suit is not None:
            value += 13*self.__SUITS.index(suit) - 1
        
        self.__value = value
        self.__face = value % 13 + 1
        self.__suit = value//13
        return
    
    def __validate_input(self, 
                         value: int, 
                         suit:  str, 
                         face:  str
                        ) -> None:
        
        MSG_BAD_INPUT_COMBO = 'Either value, OR suit and face, should be provided.'
        
        if (value is None) and (suit is None) and (face is None):
            raise ValueError(MSG_BAD_INPUT_COMBO)
        
        if value is not None:
            if (suit is not None) or (face is not None):
                raise ValueError(MSG_BAD_INPUT_COMBO)
        
        if suit is not None:
            if suit not in self.__SUITS:
                raise ValueError(suit + ' is not a valid suit name.')
            if face is None:
                raise ValueError('Suit was specified without a face value.')
        if face is not None:
            if suit is None:
                raise ValueError('Face was specified without a suit.')
        
        if (suit is not None) and (face is not None):
            if value is not None:
                raise ValueError(MSG_BAD_INPUT_COMBO)
        
        return
    
    def get_face_num(self):
        return self.__face
    
    def get_face_name(self):
        return self.__FACES[self.__face]
    
    def get_suit_num(self):
        return self.__suit
    
    def get_suit_name(self):
        return self.__SUITS[self.__suit]
    
    def __repr__(self):
        return str(self.__FACES[self.__face]) + \
               ' of ' + str(self.__SUITS[self.__suit])
    
    def __gt__(self, other):
        return self.__face > other.__face
    
    def __sub__(self, other):
        return self.__face - other.__face
    
    def __eq__(self, other):
        return (self.__face == other.__face) and (self.__suit == other.__suit)

In [2]:
# Hands to test
straight1 = Hand(
            [Card(face= 'Ace', suit = 'diamonds'),
             Card(face= '7', suit = 'diamonds'),
             Card(face= '3', suit = 'diamonds'),
             Card(face= '4', suit = 'diamonds'),
             Card(face= '5', suit = 'diamonds'),
             Card(face= '6', suit = 'diamonds'),
             Card(face= '2', suit = 'diamonds'),
            ])

pairs1 = Hand(
            [Card(face= 'Ace', suit = 'diamonds'),
             Card(face= 'Ace', suit = 'clubs'),
             Card(face= '3', suit = 'diamonds'),
             Card(face= '4', suit = 'diamonds'),
             Card(face= '5', suit = 'diamonds'),
             Card(face= '6', suit = 'diamonds'),
             Card(face= '2', suit = 'diamonds'),
            ])

pairs2 = Hand(
            [Card(face= 'Ace', suit = 'diamonds'),
             Card(face= 'Ace', suit = 'clubs'),
             Card(face= '3', suit = 'diamonds'),
             Card(face= '3', suit = 'clubs'),
             Card(face= '5', suit = 'diamonds'),
             Card(face= '5', suit = 'clubs'),
             Card(face= '2', suit = 'diamonds'),
            ])

three1 = Hand(
            [Card(face= 'Ace', suit = 'diamonds'),
             Card(face= 'Ace', suit = 'clubs'),
             Card(face= 'Ace', suit = 'spades'),
             Card(face= '3', suit = 'clubs'),
             Card(face= '5', suit = 'diamonds'),
             Card(face= '5', suit = 'clubs'),
             Card(face= '5', suit = 'spades'),
            ])

flush1 = Hand(
            [Card(face= 'Ace', suit = 'diamonds'),
             Card(face= '2', suit = 'diamonds'),
             Card(face= 'Queen', suit = 'diamonds'),
             Card(face= '3', suit = 'diamonds'),
             Card(face= '5', suit = 'diamonds'),
             Card(face= '5', suit = 'clubs'),
             Card(face= '5', suit = 'spades'),
            ])


not_straight_flush = Hand(
                    [Card(face= 'Ace', suit = 'diamonds'),
                     Card(face= '2', suit = 'diamonds'),
                     Card(face= '3', suit = 'spades'),
                     Card(face= '4', suit = 'spades'),
                     Card(face= '5', suit = 'spades'),
                     Card(face= '10', suit = 'spades'),
                     Card(face= 'Jack', suit = 'spades'),
                    ])

straight_flush1 = Hand(
                    [Card(face= 'Ace', suit = 'diamonds'),
                     Card(face= '2', suit = 'spades'),
                     Card(face= '3', suit = 'spades'),
                     Card(face= '4', suit = 'spades'),
                     Card(face= '5', suit = 'spades'),
                     Card(face= '6', suit = 'spades'),
                     Card(face= '7', suit = 'clubs'),
                    ])

straight_flush2 = Hand(
                    [Card(face= 'Ace', suit = 'spades'),
                     Card(face= '2', suit = 'spades'),
                     Card(face= '3', suit = 'spades'),
                     Card(face= '4', suit = 'spades'),
                     Card(face= '5', suit = 'spades'),
                     Card(face= '10', suit = 'spades'),
                     Card(face= 'Jack', suit = 'clubs'),
                    ])

In [3]:
straight1.get_straight()

7

In [4]:
pairs1.get_pairs()

[1]

In [5]:
pairs2.get_pairs()

[1, 3, 5]

In [6]:
pairs2.get_high_card()

5

In [7]:
three1.get_triples()

[1, 5]

In [8]:
three1.get_pairs()

[]

In [9]:
flush1.get_flush()

(12, 'Diamonds')

In [10]:
not_straight_flush.get_flush()

(11, 'Spades')

In [11]:
not_straight_flush.get_straight()

5

In [ ]:
[1 1 1 1 1 1]

In [ ]:
[1 0 1 1 1 1]

---

Random code scraps below

In [2]:
# How many hands need to be dealt, on average
# to get 4 of a kind?

if False:
    trials = 50
    num_hands = []

    for _ in range(trials):
        d = Deck()
        my_hand = d.deal(5)
        n = 1
        while len(my_hand.get_quadruples()) == 0:
            d.shuffle()
            my_hand = d.deal(5)
            n += 1
        num_hands.append(n)

    print(sum(num_hands)/trials)

In [3]:
from IPython.display import clear_output

if False:
    d = Deck()
    my_hand = d.deal(7)
    n = 1

    face, suit = my_hand.get_flush()

    while face is None:
        clear_output()
        print(n)
        d.shuffle()
        my_hand = d.deal(7)
        face, suit = my_hand.get_flush()
        n += 1

    print(my_hand)
    print(face, suit)

In [ ]:
# 1 2 3 4 5 9 10

In [ ]:
#  1 1 1 1 4 1 

In [43]:
d = Deck()
my_hand = d.deal(5)

In [44]:
my_hand.get_straight()

[3, 1, 1, 2]

In [45]:
my_hand._Hand__cards_sorted

[2 of Clubs, 5 of Hearts, 6 of Hearts, 7 of Hearts, 9 of Spades]

In [11]:
x = [2,1,1,1,1,5] # yes straight
y = [1,1,3,1,1,2] # no straight
z = [1,0,1,1,1,2] # yes straight
u = [1,1,1,1,1,1] # yes straight, index 5

In [12]:
def check(values):
    # Four occurences of a 1, after dropping zeros
    ones = 0
    cur_idx = -1
    for idx,val in enumerate(values):
        if val == 1:
            ones += 1
            if ones >= 4:
                cur_idx = idx
        elif (val == 0):
            pass
        else:
            ones = 0
    return cur_idx

In [13]:
check(x)

4

In [14]:
check(y)

-1

In [15]:
check(z)

4

In [16]:
check(u)

5